In [5]:
%%javascript
element.append('<a href="file">hello</a>')


<IPython.core.display.Javascript object>

In .local/share/jupyter/nbextensions/share_session/main.js:

```
require([
    'jquery',
    'base/js/utils',
    'services/sessions/session'
], function(
    $,
    utils,
    session
    ) {
    "use strict";
    var save_prototype_get_model = session.Session.prototype._get_model;
    session.Session.prototype._get_model = function () {
        var model = save_prototype_get_model.call(this);
        model.path= utils.get_body_data("sessionPath");
        return model;
    };
})
```


In .jupyter/jupyter_nbconvert_config.json:

    "extra_template_paths": [
      ".",
      "/home/wayne/jupyter_contrib_nbextensions_dev/src/jupyter_contrib_nbextensions/templates"
    ],



In /home/wayne/jupyter_contrib_nbextensions_dev/src/jupyter_contrib_nbextensions/templates/sharesession.html:

```
{% extends "notebook.html" %}

{% block stylesheet %} {{super()}} {% endblock %}

{% block bodyclasses %}notebook_app {{super()}}{% endblock %}

{% block params %} {{super()}} {% endblock %}

{% block headercontainer %} {{super()}} {% endblock headercontainer %}

{% block header %} {{super()}} {% endblock header %}

{% block site %} {{super()}} {% endblock %}

{% block after_site %} {{super()}} {% endblock %}

{% block script %}

<script src="/nbextensions/share_session/main.js" type="text/javascript" charset="utf-8"></script>
{{super()}}

{% endblock %}
```

In `<pythonpath>`/sharesession.py:

```
import os
from tornado import web
HTTPError = web.HTTPError

from notebook.base.handlers import (
    IPythonHandler, FilesRedirectHandler, path_regex,
)
from notebook.utils import url_escape


class ShareSessionHandler(IPythonHandler):
    @web.authenticated
    def get(self, path):
        """get renders the notebook template if a name is given, or 
        redirects to the '/files/' handler if the name is not given."""
        session_path = self.get_argument('sharesession',None)
        if session_path is None:
            session_path = path.rsplit('/', 1)[0]+'/index.ipynb'
        path = path.strip('/')
        cm = self.contents_manager
        
        # will raise 404 on not found
        try:
            model = cm.get(path, content=False)
        except web.HTTPError as e:
            if e.status_code == 404 and 'files' in path.split('/'):
                # 404, but '/files/' in URL, let FilesRedirect take care of it
                return FilesRedirectHandler.redirect_to_files(self, path)
            else:
                raise
        if model['type'] != 'notebook':
            # not a notebook, redirect to files
            return FilesRedirectHandler.redirect_to_files(self, path)
        name = path.rsplit('/', 1)[-1]
        self.write(self.render_template('sharesession.html',
            notebook_path=path,
            session_path=session_path,
            notebook_name=name,
            kill_kernel=False,
            mathjax_url=self.mathjax_url,
            mathjax_config=self.mathjax_config
            )
        )


from notebook.utils import url_path_join
def load_jupyter_server_extension(nb_server_app):
    """
    Called when the extension is loaded.

    Args:
        nb_server_app (NotebookWebApplication): handle to the Notebook webserver instance.
    """
    web_app = nb_server_app.web_app
    host_pattern = '.*$'
    route_pattern = url_path_join(web_app.settings['base_url'], '/sharesession%s' % path_regex)
    web_app.add_handlers(host_pattern, [(route_pattern, ShareSessionHandler)])

```